In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
eps = 1e-9

In [ ]:
probs = torch.tensor([[0.04, 0.05, 0.04, 0.15, 0.11, 0.1, 0.11, 0.2, 0.1, 0.1],
                      [0.03, 0.1, 0.2, 0.02, 0.12, 0.1, 0.13, 0.1, 0.1, 0.1],
                      [0.02, 0.1, 0.05, 0.4, 0.13, 0.1, 0.0, 0.1, 0.04, 0.06]])

In [ ]:
zeros = torch.zeros(3, 6)

In [ ]:
_, pred = torch.max(probs, dim=1)
pred

In [ ]:
labels = torch.tensor([[3, 1, 0, 7, 4, 0], [0, 0, 0, 0, 0, 0], [2, 8, 9, 0, 0, 0]])
labels

In [ ]:
exist_label = labels.sum(dim=1).float().ge(0.1)
exist_label

In [ ]:
selected_labels = torch.where(labels == pred.view(-1, 1), labels, zeros.long())
selected_labels

In [ ]:
unselected_labels = torch.where(labels == pred.view(-1, 1), zeros.long(), labels)
unselected_labels

In [ ]:
next_labels = unselected_labels

In [ ]:
selected_probs = torch.gather(probs, 1, selected_labels).float()
selected_probs

In [ ]:
unselected_probs = torch.gather(probs, 1, unselected_labels).float()
unselected_probs

In [ ]:
selected_loss = -torch.log(selected_probs + eps)
selected_loss

In [ ]:
unselected_loss = -torch.log(unselected_probs + eps)
unselected_loss

In [ ]:
selected_loss = torch.where(selected_labels == torch.zeros(1).long(), zeros, selected_loss)
selected_loss

In [ ]:
unselected_loss = torch.where(unselected_labels == torch.zeros(1).long(), zeros, unselected_loss)
unselected_loss

In [ ]:
selected_loss, _ = torch.max(selected_loss, dim=1)
selected_loss

In [ ]:
unselected_loss = unselected_loss.sum(dim=1) / (unselected_labels.float().ge(0.5).sum(dim=1).float() + eps)
unselected_loss

In [ ]:
exist_selected_label = selected_labels.float().sum(dim=1).ge(0.5)
exist_selected_label

In [ ]:
mean_loss = torch.where(exist_selected_label == torch.ones(1).byte(), selected_loss, unselected_loss)
mean_loss

In [ ]:
mean_loss = mean_loss.masked_select(exist_label)
mean_loss

In [ ]:
mean_loss = mean_loss.sum()
mean_loss